In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [3]:
dong_sw = pd.read_csv('file/year_all.csv')

In [4]:
dong_sw

,행정동,시군구,총_승객수,연분기
0,가락1동,송파구,18745.12,20221
1,가락2동,송파구,10442.12,20221
2,가산동,금천구,38186.30,20221
3,갈현1동,은평구,29402.29,20221
4,갈현2동,은평구,5997.46,20221
...,...,...,...,...
4190,화곡본동,강서구,0.00,20242
4191,황학동,중구,0.00,20242
4192,회기동,동대문구,0.00,20242
4193,후암동,용산구,0.00,20242
